About the Dataset:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks whether the news article is real or fake

    1: Fake news
    0: real News

In [70]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\J.S. CAITLYN
[nltk_data]     MARY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# Load datasets
df1 = pd.read_csv('news.csv')
df2 = pd.read_csv('IFND.csv', encoding='latin1')
df3 = pd.read_csv('news_dataset.csv', encoding='latin1')


In [6]:
# Rename columns
df2 = df2.rename(columns={'Statement': 'text', 'Label': 'label'})

# Concatenate DataFrames
news_dataset = pd.concat([df1[['text', 'label']], df2[['text', 'label']], df3[['text', 'label']]], ignore_index=True)

# Drop duplicates and NaN values
news_dataset.drop_duplicates(subset='text', inplace=True)
news_dataset.dropna(inplace=True)

In [7]:
news_dataset.shape

(64629, 2)

In [8]:
# print the first 5 rows of the dataframe
news_dataset.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [9]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

text     0
label    0
dtype: int64

In [10]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [11]:
# separating the data & label
X = news_dataset['text']
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

0        Daniel Greenfield, a Shillman Journalism Fello...
1        Google Pinterest Digg Linkedin Reddit Stumbleu...
2        U.S. Secretary of State John F. Kerry said Mon...
3        — Kaydee King (@KaydeeKing) November 9, 2016 T...
4        It's primary day in New York and front-runners...
                               ...                        
66768    A set of images is being shared on Facebook wi...
66769    Barely 48 hours ahead of voting in the assembl...
66771    A quote by an impostor Facebook page of Financ...
66775    The Bengaluru City Policeâs official Twitter...
66776    Sep 20, 2020, 08:00AM IST\n\nSource: TOI.in\n\...
Name: text, Length: 64629, dtype: object
0        FAKE
1        FAKE
2        REAL
3        FAKE
4        REAL
         ... 
66768    FAKE
66769    FAKE
66771    FAKE
66775    FAKE
66776    REAL
Name: label, Length: 64629, dtype: object


Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(text):
    stemmed_text = re.sub('[^a-zA-Z]',' ',text)
    stemmed_text = stemmed_text.lower()
    stemmed_text = stemmed_text.split()
    stemmed_text = [port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
    stemmed_text = ' '.join(stemmed_text)
    return stemmed_text

In [15]:
news_dataset['text'] = news_dataset['text'].apply(stemming)

In [16]:
#separating the data and label
X = news_dataset['text'].values
Y = news_dataset['label'].values

In [17]:
print(X)

['daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scandal realli attack 

In [18]:
Y.shape

(64629,)

In [19]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

Splitting the dataset to training & test data

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

### Training the Models: 

### (1) Logistic Regression

In [21]:
model = LogisticRegression()

In [22]:
model.fit(X_train, Y_train)

c:\Fakenewsdetection\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [23]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [24]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9505831383091891


In [25]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [26]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9353241528701841


In [27]:
# Classification report on test data
print("Classification Report:\n", classification_report(Y_test, X_test_prediction))

Classification Report:
               precision    recall  f1-score   support

        FAKE       0.94      0.88      0.91       985
        Fake       0.97      0.86      0.91      3708
        REAL       0.92      0.85      0.89       673
        TRUE       0.92      0.99      0.95      7560

    accuracy                           0.94     12926
   macro avg       0.94      0.89      0.91     12926
weighted avg       0.94      0.94      0.93     12926



### (2) Navie Byes

In [28]:
from sklearn.naive_bayes import MultinomialNB


In [29]:
# Initialize the Naive Bayes Classifier
model_nb = MultinomialNB()

In [30]:
# Fit the model
model_nb.fit(X_train, Y_train)


MultinomialNB()

In [31]:
# Accuracy score on the training data
X_train_prediction = model_nb.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data (Naive Bayes): ', training_data_accuracy)

Accuracy score of the training data (Naive Bayes):  0.9060596096938283


In [32]:
# Accuracy score on the test data
X_test_prediction = model_nb.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data (Naive Bayes): ', test_data_accuracy)

Accuracy score of the test data (Naive Bayes):  0.8886739904069317


In [33]:
# Classification report on test data
print("Classification Report (Naive Bayes):\n", classification_report(Y_test, X_test_prediction))

Classification Report (Naive Bayes):
               precision    recall  f1-score   support

        FAKE       0.89      0.62      0.73       985
        Fake       0.92      0.79      0.85      3708
        REAL       0.89      0.75      0.82       673
        TRUE       0.88      0.99      0.93      7560

    accuracy                           0.89     12926
   macro avg       0.89      0.79      0.83     12926
weighted avg       0.89      0.89      0.88     12926



### (3) PassiveAggressive Classifier

In [34]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [35]:
pmodel = PassiveAggressiveClassifier(max_iter=1000, random_state=42)

In [36]:
pmodel.fit(X_train, Y_train)

# Accuracy score on the training data
X_train_prediction = pmodel.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data : ', training_data_accuracy)


Accuracy score of the training data :  0.9913351256213373


In [37]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data : ', test_data_accuracy)


Accuracy score of the test data :  0.9353241528701841


### (4)Random Tree

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
# Initialize the Random Forest Classifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42) 

In [40]:
# Fit the model
model_rf.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [41]:
# Accuracy score on the training data
X_train_prediction = model_rf.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data (Random Forest): ', training_data_accuracy)

Accuracy score of the training data (Random Forest):  0.9971568380945013


In [42]:
# Accuracy score on the test data
X_test_prediction = model_rf.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data (Random Forest): ', test_data_accuracy)

Accuracy score of the test data (Random Forest):  0.9378771468358348


In [65]:
X_new = X_test[3].reshape(1, -1)  # Reshape for a single sample
prediction = model_rf.predict(X_new)
print("Prediction for X_new:", prediction)

if prediction[0] == 0:
    print('The news is Fake')
else:
    print('The news is Real')

# Check actual label
print('Actual label:', Y_test[3])

Prediction for X_new: ['TRUE']
The news is Real
Actual label: TRUE


In [43]:
# Classification report on test data
print("Classification Report (Random Forest):\n", classification_report(Y_test, X_test_prediction))

Classification Report (Random Forest):
               precision    recall  f1-score   support

        FAKE       0.84      0.88      0.86       985
        Fake       0.96      0.90      0.93      3708
        REAL       0.90      0.75      0.82       673
        TRUE       0.95      0.98      0.96      7560

    accuracy                           0.94     12926
   macro avg       0.91      0.88      0.89     12926
weighted avg       0.94      0.94      0.94     12926



### (5)Decision Tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

In [45]:
# Initialize the Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=42)

In [46]:
# Fit the model
model_dt.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=42)

In [47]:
# Accuracy score on the training data
X_train_prediction = model_dt.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data (Decision Tree): ', training_data_accuracy)

Accuracy score of the training data (Decision Tree):  0.9971568380945013


In [48]:

# Accuracy score on the test data
X_test_prediction = model_dt.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data (Decision Tree): ', test_data_accuracy)

Accuracy score of the test data (Decision Tree):  0.9086337614111094


In [49]:
# Classification report on test data
print("Classification Report (Decision Tree):\n", classification_report(Y_test, X_test_prediction))

Classification Report (Decision Tree):
               precision    recall  f1-score   support

        FAKE       0.75      0.81      0.78       985
        Fake       0.90      0.90      0.90      3708
        REAL       0.76      0.75      0.76       673
        TRUE       0.95      0.94      0.94      7560

    accuracy                           0.91     12926
   macro avg       0.84      0.85      0.85     12926
weighted avg       0.91      0.91      0.91     12926



### (6) KNN

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
model_knn = KNeighborsClassifier(n_neighbors=5)

In [52]:

# Fit the model
model_knn.fit(X_train, Y_train)

KNeighborsClassifier()

In [53]:

# Accuracy score on the training data
X_train_prediction = model_knn.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data (KNN): ', training_data_accuracy)

Accuracy score of the training data (KNN):  0.8059493646403497


In [54]:

# Accuracy score on the test data
X_test_prediction = model_knn.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data (KNN): ', test_data_accuracy)

Accuracy score of the test data (KNN):  0.7226520191861364


In [55]:
# Classification report on test data
print("Classification Report (KNN):\n", classification_report(Y_test, X_test_prediction))

Classification Report (KNN):
               precision    recall  f1-score   support

        FAKE       0.21      0.80      0.33       985
        Fake       0.87      0.75      0.81      3708
        REAL       0.92      0.75      0.83       673
        TRUE       0.97      0.69      0.81      7560

    accuracy                           0.72     12926
   macro avg       0.74      0.75      0.69     12926
weighted avg       0.88      0.72      0.77     12926



### (7)SVM

In [56]:
from sklearn.svm import SVC

In [57]:
# Initialize the Support Vector Classifier (SVM)
model_svm = SVC(kernel='linear', random_state=42)

In [58]:
# Fit the model
model_svm.fit(X_train, Y_train)

SVC(kernel='linear', random_state=42)

In [59]:
# Accuracy score on the training data
X_train_prediction = model_svm.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of the training data (SVM): ', training_data_accuracy)

Accuracy score of the training data (SVM):  0.963522426164826


In [60]:
# Accuracy score on the test data
X_test_prediction = model_svm.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of the test data (SVM): ', test_data_accuracy)

Accuracy score of the test data (SVM):  0.9436020423951725


In [61]:

# Classification report on test data
print("Classification Report (SVM):\n", classification_report(Y_test, X_test_prediction))

Classification Report (SVM):
               precision    recall  f1-score   support

        FAKE       0.95      0.92      0.94       985
        Fake       0.97      0.87      0.92      3708
        REAL       0.93      0.89      0.91       673
        TRUE       0.93      0.99      0.96      7560

    accuracy                           0.94     12926
   macro avg       0.95      0.92      0.93     12926
weighted avg       0.94      0.94      0.94     12926



In [68]:
with open('model__rf', 'wb') as file:
    pickle.dump(model, file) 

In [69]:
with open('my__vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file) 